In [ ]:
import time
from v3_Micro.PradaQ_Live import PradaQ_Live
from v3_Micro.OrderManager import OrderManager
from v3_Micro.MasterDashboard2 import MasterDashboard2
from v3_Micro.MasterDashboard2 import PAIRS_WATCHLIST

# 1. INITIALIZE (The Fund)
all_tickers = list(set([p['a'] for p in PAIRS_WATCHLIST] + [p['b'] for p in PAIRS_WATCHLIST]))
prod_engine = PradaQ_Live(all_tickers)
prod_engine.ingest_and_clean(start="2020-01-01", end="2025-01-01")

# 2. SETUP METADATA (The Anchors)
pair_metadata = {}
for p in PAIRS_WATCHLIST:
    p_id = f"{p['a']}_{p['b']}"
    prod_engine.anchor_pair_statistics(p['a'], p['b'])
    pair_metadata[p_id] = {
        'beta': prod_engine.anchor_beta,
        'mean': prod_engine.historical_mean,
        'std': prod_engine.historical_std,
        'manager': OrderManager(capital=100000)
    }

# 3. RUN MASTER HEARTBEAT
dash = MasterDashboard2()
for ts, row in prod_engine.data.tail(200).iterrows():
    pair_results = {}
    grand_total_pnl = 0
    
    for p in PAIRS_WATCHLIST:
        p_id = f"{p['a']}_{p['b']}"
        meta = pair_metadata[p_id]
        
        # Math & Signal
        spread = row[p['a']] - (meta['beta'] * row[p['b']])
        z = (spread - meta['mean']) / meta['std']
        signal = 1 if z < -1.0 else (-1 if z > 1.0 else 0)
        
        # Execution
        meta['manager'].execute_paper_trade(signal, row[p['a']], row[p['b']], meta['beta'], z)
        
        pair_results[p_id] = {'z': z, 'pos': meta['manager'].position}
        grand_total_pnl += meta['manager'].total_pnl
        
    dash.update(pair_results, grand_total_pnl)
    time.sleep(0.05)

 PROJECT NEXUS: MULTI-PAIR PRODUCTION MONITOR 
PAIR                         | Z-SCORE  | POS     
-------------------------------------------------------
HDFCBANK.NS_ICICIBANK.NS     |    0.01 | 0       
  [----------O----------]
RPOWER.BO_ADANIPOWER.NS      |    3.19 | -1      
  [----------|---------O]
APOLLOHOSP.NS_CIPLA.NS       |    0.65 | 0       
  [----------|-O--------]
INFY.NS_HCLTECH.NS           |   -0.92 | 0       
  [------O---|----------]
TATASTEEL.NS_JSWSTEEL.NS     |   -0.90 | 0       
  [-------O--|----------]
-------------------------------------------------------
TOTAL PORTFOLIO PNL: ₹34.77


In [2]:
print("\n" + "="*30)
print(" FINAL STRATEGY AUTOPSY ")
print("="*30)
for p_id, meta in pair_metadata.items():
    pnl = meta['manager'].total_pnl
    status = "PROFIT" if pnl > 0 else "LOSS"
    print(f"{p_id:<25}: ₹{pnl:>8.2f} [{status}]")

print("-" * 30)
print(f"NET PERFORMANCE: ₹{grand_total_pnl:.2f}")
print(f"EFFICIENCY (PnL per Pair): ₹{grand_total_pnl/len(PAIRS_WATCHLIST):.2f}")
print("="*30)


 FINAL STRATEGY AUTOPSY 
HDFCBANK.NS_ICICIBANK.NS : ₹    0.00 [LOSS]
RPOWER.BO_ADANIPOWER.NS  : ₹   12.58 [PROFIT]
APOLLOHOSP.NS_CIPLA.NS   : ₹    0.00 [LOSS]
INFY.NS_HCLTECH.NS       : ₹    0.00 [LOSS]
TATASTEEL.NS_JSWSTEEL.NS : ₹   22.19 [PROFIT]
------------------------------
NET PERFORMANCE: ₹34.77
EFFICIENCY (PnL per Pair): ₹6.95
